# Grupper patienter med unsupervised clustering med k-means

https://www.kaggle.com/competitions/k-means-clustering-for-heart-disease-analysis/data

## Hent data fra kaggle og put det i dataframes

In [45]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

heart_disease_csv = pd.read_csv('data/heart_disease.csv')
sample_csv = pd.read_csv('data/sample.csv')

heart_disease_csv.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal
0,0,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect
1,1,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal
2,2,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect
3,3,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal
4,4,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal


## Filtrer dataen så jeg kun bruger patienter som har et id fra sample filen. (Det var en del af opgaven)

In [46]:
valid_ids = set(sample_csv['id'])
data = heart_disease_csv[heart_disease_csv['id'].isin(valid_ids)]

print(data['id'].count())

299


## Gem dataen i ny csv

fjern alle rows med nan data, da det vil ødelægge vores resultater. Jeg har tænkt at der måske er bedre måder at gøre det på. måske at bare fjerne den colonne med mange nan, eller på en måde fylde det ud med mock data af samme statiske frekvens som fyldt data, Dette kaldes for Mean Imputation. Men jeg føler der er for mange faktorer i dataen til at jeg kan gøre det, så som hvis de data man fylder ind har en direkte relation til noget andet data, og det ikke bliver reflekteret i den tilfældigt indsatte data. Da vi har med medicinsk data, tænker jeg at det er yderst vigtigt at data'en er korrekt. At indsætte noget tilfældig data ville ikke være hensigtsmæssigt i denne sammenhæng, også selv om vi får betydeligt færre samples.

dropper også id da den ikke har betydning for dataen.

In [47]:
data = data.dropna()
data = data.drop('id', axis=1)

data.to_csv('data/heart_disease_filtered.csv', index=False)

print(data['age'].count())

109
